# Keras-CNN
参见该文：[Keras Model for Beginners (0.210 on LB)+EDA+R&D
](https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d)

In [1]:
import numpy as np
import pandas as pd

from subprocess import check_output
#print(check_output(["ls", "../Data"]).decode("utf8"))

#print(check_output(["ls", "/kaggle/ShipIceberg/Data"]).decode("utf8"))

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [3]:
#Load the data.
train = pd.read_json("../ShipIceberg/Data/train.json")
# test = pd.read_json("../ShipIceberg/Data/test.json")

直接用read_json显示Could not reserve memory block

In [4]:
#test = pd.read_json("../ShipIceberg/Data/test.json")
import json
with open('../ShipIceberg/Data/test.json', 'r') as f:
    test = json.load(f)
    test=pd.DataFrame(test)
    
print('done!')

done!


In [5]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
# test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_test_angle=test['inc_angle']

In [6]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])


X_band_3 = np.zeros((1604,75,75))
for i in range(0,1604):
    subt = abs(X_band_1[i]-X_band_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_3[i]=W1 * X_band_1[i]+W2 * X_band_2[i]



X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)


#Generate the test data
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])


X_band_test_3 = np.zeros((8424,75,75))
for i in range(0,8424):
    subt = abs(X_band_test_1[i]-X_band_test_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_test_3[i]=W1 * X_band_test_1[i]+W2 * X_band_test_2[i]



X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

del train;del test;

In [7]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

Using TensorFlow backend.


In [8]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    
        # 使用不同的优化
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    adagrad = Adagrad(lr = 1e-3, epsilon = 1e-6)
    rmsprop = RMSprop(lr=1e-3, rho = 0.9, epsilon=1e-6)
    adadelta = Adadelta(lr=1e-3, rho=0.95, epsilon=1e-06)
    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    adamax = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

    mypotim=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
#     gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
# file_path = "./model/model_weights.hdf5"
# callbacks = get_callbacks(filepath=file_path, patience=5)

# aug1
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
#                          samplewise_center=True,
#                          samplewise_std_normalization=True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0.,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_twelve_inputs(X1, 
#                                X2, X3,X4,X5,X6,X7,X8,X9,X10,X11,X12, 
                               y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    '''
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    genX3 = gen.flow(X1,X3, batch_size=batch_size,seed=55)
    genX4 = gen.flow(X1,X4, batch_size=batch_size,seed=55)
    genX5 = gen.flow(X1,X5, batch_size=batch_size,seed=55)
    genX6 = gen.flow(X1,X6, batch_size=batch_size,seed=55)
    genX7 = gen.flow(X1,X7, batch_size=batch_size,seed=55)
    genX8 = gen.flow(X1,X8, batch_size=batch_size,seed=55)
    genX9 = gen.flow(X1,X9, batch_size=batch_size,seed=55)
    genX10 = gen.flow(X1,X10, batch_size=batch_size,seed=55)
    genX11 = gen.flow(X1,X11, batch_size=batch_size,seed=55)
    genX12 = gen.flow(X1,X12, batch_size=batch_size,seed=55)
#     genX13 = gen.flow(X1,X13, batch_size=batch_size,seed=55)
'''
    while True:
            X1i = genX1.next()
            '''
            X2i = genX2.next()
            X3i = genX3.next()
            X4i = genX4.next()
            X5i = genX5.next()
            X6i = genX6.next()
            X7i = genX7.next()
            X8i = genX8.next()
            X9i = genX9.next()
            X10i = genX10.next()
            X11i = genX11.next()
            X12i = genX12.next()
#             X13i = genX13.next()
'''
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], 
#                    X2i[1],X3i[1],X4i[1],X5i[1],X6i[1],X7i[1],
#                   X8i[1],X9i[1],X10i[1],X11i[1],X12i[1]
                  ],  X1i[1]


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_test):
    # K-折交叉验证
    K=10
    
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        '''
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        X_min1_cv=X_min1[train_idx]
        X_min1_hold=X_min1[test_idx]
        
        X_max1_cv=X_max1[train_idx]
        X_max1_hold=X_max1[test_idx]
        
        X_std1_cv=X_std1[train_idx]
        X_std1_hold=X_std1[test_idx]
        
        X_med1_cv=X_med1[train_idx]
        X_med1_hold=X_med1[test_idx]
        
        X_mean1_cv=X_mean1[train_idx]
        X_mean1_hold=X_mean1[test_idx]
        
        X_min2_cv=X_min2[train_idx]
        X_min2_hold=X_min2[test_idx]
        
        X_max2_cv=X_max2[train_idx]
        X_max2_hold=X_max2[test_idx]
        
        X_std2_cv=X_std2[train_idx]
        X_std2_hold=X_std2[test_idx]
        
        X_med2_cv=X_med2[train_idx]
        X_med2_hold=X_med2[test_idx]
        
        X_mean2_cv=X_mean2[train_idx]
        X_mean2_hold=X_mean2[test_idx]
        '''
        
        
        #define file path and get callbacks
        file_path = "./model/%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_twelve_inputs(X_train_cv, 
#                                               X_angle_cv, 
#                                               X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
#                                               X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv,
                                              y_train_cv)
#         galaxyModel= getVggAngleModel()
        galaxyModel= getModel()
        
        # 调整训练参数
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                #steps_per_epoch=100,
                epochs=100,
#                 shuffle=True,
                verbose=1,
                validation_data=([X_holdout],
#                                   X_angle_hold,
#                                  X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
#                                  X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold], 
                                 Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate(X_train_cv,
#                                       X_angle_cv,
#                                      X_min1_cv,X_max1_cv,X_std1_cv,X_med1_cv,X_mean1_cv,
#                                      X_min2_cv,X_max2_cv,X_std2_cv,X_med2_cv,X_mean2_cv], 
                                     y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate(X_holdout,
#                                      X_angle_hold,
#                                      X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
#                                      X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold], 
                                     Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,
#                                         X_angle_hold,
#                                        X_min1_hold,X_max1_hold,X_std1_hold,X_med1_hold,X_mean1_hold,
#                                        X_min2_hold,X_max2_hold,X_std2_hold,X_med2_hold,X_mean2_hold
                                       ])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, 
#                                        X_test_angle,
#                                       X_test_min1,X_test_max1,X_test_std1,X_test_med1,X_test_mean1,
#                                       X_test_min2,X_test_max2,X_test_std2,X_test_med2,X_test_mean2
                                      ])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train,
#                                         X_angle,
#                                        X_min1,X_max1,X_std1,X_med1,X_mean1,
#                                        X_min2,X_max2,X_std2,X_med2,X_mean2
                                       ])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [9]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import log_loss
from keras.optimizers import Adam
#from keras.optimizers import rmsprop
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import Adamax
from keras.optimizers import Nadam

batch_size = 64

preds=myAngleCV(X_train, X_test)
#                 X_angle, 
#                 X_min1,X_max1,X_std1,X_med1,X_mean1,
#                 X_min2,X_max2,X_std2,X_med2,X_mean2,
                


===================FOLD= 0
Epoch 1/100
24/24 [==============================] - 3s - loss: 1.0629 - acc: 0.5151 - val_loss: 0.6915 - val_acc: 0.5247
Epoch 2/100
24/24 [==============================] - 1s - loss: 0.6943 - acc: 0.5326 - val_loss: 0.6847 - val_acc: 0.6111
Epoch 3/100
24/24 [==============================] - 1s - loss: 0.6806 - acc: 0.5579 - val_loss: 0.6752 - val_acc: 0.5741
Epoch 4/100
24/24 [==============================] - 1s - loss: 0.6669 - acc: 0.5431 - val_loss: 0.6625 - val_acc: 0.6667
Epoch 5/100
24/24 [==============================] - 1s - loss: 0.6428 - acc: 0.5851 - val_loss: 0.6398 - val_acc: 0.6481
Epoch 6/100
24/24 [==============================] - 1s - loss: 0.6329 - acc: 0.5777 - val_loss: 0.6253 - val_acc: 0.6605
Epoch 7/100
24/24 [==============================] - 2s - loss: 0.6123 - acc: 0.5814 - val_loss: 0.6070 - val_acc: 0.6481
Epoch 8/100
24/24 [==============================] - 2s - loss: 0.6042 - acc: 0.6105 - val_loss: 0.5967 - val_acc: 0.6

24/24 [==============================] - 1s - loss: 0.5353 - acc: 0.7088 - val_loss: 0.5069 - val_acc: 0.7702
Epoch 17/100
24/24 [==============================] - 1s - loss: 0.5538 - acc: 0.7036 - val_loss: 0.4991 - val_acc: 0.7702
Epoch 18/100
24/24 [==============================] - 1s - loss: 0.5257 - acc: 0.7326 - val_loss: 0.4857 - val_acc: 0.7888
Epoch 19/100
24/24 [==============================] - 1s - loss: 0.5292 - acc: 0.7080 - val_loss: 0.4874 - val_acc: 0.7702
Epoch 20/100
24/24 [==============================] - 1s - loss: 0.5218 - acc: 0.7257 - val_loss: 0.4725 - val_acc: 0.7950
Epoch 21/100
24/24 [==============================] - 2s - loss: 0.5067 - acc: 0.7437 - val_loss: 0.4599 - val_acc: 0.8075
Epoch 22/100
22/24 [==========================>...] - ETA: 0s - loss: 0.4950 - acc: 0.7511

KeyboardInterrupt: 

In [ ]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('subKerasCNN1.csv', index=False)